In [2]:
import pandas as pd
import sqlite3

df = pd.read_csv(r'C:\Python\Setup\.venv\FinalModels\Model2\Data\adult.csv')

conn = sqlite3.connect(r'C:\Python\Setup\.venv\FinalModels\Model2\Database\model1.db')

df.to_sql('adult_data', conn, if_exists='replace', index=False)

conn.commit()
conn.close()




In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import sqlite3  


conn = sqlite3.connect(r'C:\Python\Setup\.venv\FinalModels\Model2\Database\model1.db')


query = "SELECT * FROM adult_data"
df = pd.read_sql_query(query, conn)


conn.close()

df = df.drop(['fnlwgt', 'educational-num'], axis=1)

for col in df.columns:
    df[col] = df[col].replace("?", np.NaN)
    df[col] = df[col].fillna(df[col].value_counts().index[0])

category_col = ['workclass', 'race', 'education', 'marital-status', 'occupation',
                'relationship', 'gender', 'native-country', 'income']
labelEncoder = preprocessing.LabelEncoder()
mapping_dict = {}

for col in category_col:
    df[col] = labelEncoder.fit_transform(df[col])
    le_name_mapping = dict(zip(labelEncoder.classes_, labelEncoder.transform(labelEncoder.classes_)))
    mapping_dict[col] = le_name_mapping

X = df.drop('income', axis=1)
Y = df['income']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=100)

oob_scores = []
for _ in range(10):
    rf_regressor = RandomForestRegressor(n_estimators=100, oob_score=True, random_state=100)
    rf_regressor.fit(X_train, y_train)
    oob_scores.append(rf_regressor.oob_score_)


average_oob_score = sum(oob_scores) / len(oob_scores)
print("Average Out-of-Bag (OOB) Score:", average_oob_score)


Average Out-of-Bag (OOB) Score: 0.3935300685782517


In [4]:
df

,age,workclass,education,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,7,9,4,1,1,4,1,2174,0,40,39,0
1,50,6,9,2,4,0,4,1,0,0,13,39,0
2,38,4,11,0,6,1,4,1,0,0,40,39,0
3,53,4,1,2,6,0,2,1,0,0,40,39,0
4,28,4,9,2,10,5,2,0,0,0,40,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,4,7,2,13,5,4,0,0,0,38,39,0
32557,40,4,11,2,7,0,4,1,0,0,40,39,1
32558,58,4,11,6,1,4,4,0,0,0,40,39,0
32559,22,4,11,4,1,3,4,1,0,0,20,39,0


In [6]:
import pickle 
with open(r'C:\Python\Setup\.venv\FinalModels\Model2\SavedModel\model.pkl', 'wb') as file:
    pickle.dump(rf_regressor, file)